# Workspace - Challenge 1

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

In [3]:
from lumen.io import read_events
from lumen.io import read_positions
from lumen.util import extract_device_events

from lumen.device import Device
from lumen.model_greed_cached import ModelGreedCached

## Read data

In [4]:
df_events = read_events('site_1')
df_positions = read_positions('site_1')

In [5]:
events_map = extract_device_events(df_events)
print(f"Number of mapped devices: {len(events_map.keys())}")

Number of mapped devices: 56


In [6]:
device_id_list = sorted(df_events.deviceid.unique())

train_size = int(0.1 * len(device_id_list))
train_device_id_list = np.random.choice(device_id_list, train_size, replace=False)
test_device_id_list = [x for x in device_id_list if x not in train_device_id_list]

len(train_device_id_list), len(test_device_id_list)

(5, 51)

In [7]:
devices_pos = df_positions.set_index('deviceid')

In [8]:
devices = []
real_device_id_to_position_map = {}

for device_id in device_id_list:
    point = devices_pos.loc[device_id]
    position = [point.x, point.y]
    _device = Device(position=position, device_id=None)    
    if device_id in train_device_id_list:
        # Add as known device
        _device.set_device_id(device_id)

    devices.append(_device)
    # To calculate loss
    real_device_id_to_position_map[device_id] = [point.x, point.y]

In [9]:
known_devices = [device for device in devices if device.has_device_id()]
device_id_candidates = set(test_device_id_list)

len(known_devices), len(device_id_candidates)

(5, 51)

In [ ]:
model = ModelGreedCached(devices)
model.fit(df_events)

In [ ]:
model.map_unknowns(test_device_id_list)

NameError: name 'get_dict_key_for_cache_from_ids' is not defined

In [ ]:
# Calculate loss
loss = 0
for device in devices:
    position = device.position
    real_position = real_device_id_to_position_map.get(device.device_id)
    loss += ((position[0] - real_position[0])**2 + (position[1] - real_position[1])**2)**0.5

loss

5312.234049355525

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e19bc3ac-22fd-4ca3-9f23-d099570b4b40' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>